In [4]:
from huggingface_hub import HfApi

In [5]:
api = HfApi()

In [9]:
ds = list(api.list_datasets(
    task_categories=["text-generation"]
))

In [15]:
dataset_urls = [f"https://huggingface.com/datasets/{i.id}" for i in ds]

In [17]:
import pandas as pd

df = pd.DataFrame(dataset_urls, columns=['url'])
df.to_csv('../data/hf_datasets.csv', index=False)

In [18]:
df.sample(n=100).to_csv('../data/hf_datasets_sample.csv', index=False)